In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import seaborn as sns
import matplotlib.pyplot as plt
import missingno as ms
% matplotlib inline

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/twitter-hate-speech/train_E6oV3lV.csv')
test_data = pd.read_csv('../input/twitter-hate-speech/test_tweets_anuFYb8.csv')

In [ ]:
train_data.shape, test_data.shape

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data['label'].value_counts()


In [ ]:
test_data.head()

In [ ]:
#cleaning the data

def drop_features(features,data):
    data.drop(features,inplace=True,axis=1)

In [ ]:
import re
## example ## 
re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ","ouch...junior is angryð#got7 #junior #yugyo..., @user")

In [ ]:
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

In [ ]:
train_data['processed_tweets'] = train_data['tweet'].apply(process_tweet)

In [ ]:
train_data.head(10)

In [ ]:
drop_features(['id','tweet'],train_data)

In [ ]:
train_data.info()


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data["processed_tweets"], train_data["label"], test_size = 0.2, random_state = 42)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

In [ ]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

In [ ]:
print(x_train_counts.shape)
print(x_train_tfidf.shape)

In [ ]:
x_train_counts

In [ ]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

In [ ]:
print(x_test_counts.shape)
print(x_test_tfidf.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)
model.fit(x_train_tfidf,y_train)

In [ ]:
predictions = model.predict(x_test_tfidf)

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,predictions)

In [ ]:
#tp=5904
#tn = 237
#fn = 33
#fp = 219

#precision = tp/(tp+fp)
#recall = tp/(tp+fn)
#f1score = 2 * (recall * precision) / (recall + precision)
#f1score

In [ ]:
f1_score(y_test,predictions)

In [ ]:
predictions

**preparing for test data**

In [ ]:
test_data.info()

In [ ]:
test_data['processed_tweet'] = test_data['tweet'].apply(process_tweet)

In [ ]:
test_data.head()


In [ ]:
drop_features(['tweet'],test_data)

In [ ]:
train_counts = count_vect.fit_transform(train_data['processed_tweets'])
test_counts = count_vect.transform(test_data['processed_tweet'])

In [ ]:
print(train_counts.shape)
print(test_counts.shape)

In [ ]:
train_tfidf = transformer.fit_transform(train_counts)
test_tfidf = transformer.transform(test_counts)

In [ ]:

print(train_tfidf.shape)
print(test_tfidf.shape)


In [ ]:
model.fit(train_tfidf,train_data['label'])

In [ ]:
predictions = model.predict(test_tfidf)

In [ ]:
final_result = pd.DataFrame({'id':test_data['id'],'label':predictions})
final_result.to_csv('Output.csv',index=False)

In [ ]:
final_result.head()